In [1]:
!git clone https://github.com/mseaberg/lcls_beamline_toolbox
import os
os.chdir('lcls_beamline_toolbox')
!python3 -m pip install -e .
!pip install xraydb -q

Cloning into 'lcls_beamline_toolbox'...
remote: Enumerating objects: 3330, done.
remote: Counting objects: 100% (601/601), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 3330 (delta 523), reused 452 (delta 382), pack-reused 2729 (from 1)
Receiving objects: 100% (3330/3330), 232.17 MiB | 21.80 MiB/s, done.
Resolving deltas: 100% (2573/2573), done.
Updating files: 100% (144/144), done.
Obtaining file:///content/lcls_beamline_toolbox
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 59.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 

In [2]:
!pip install xopt==2.3.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typegu

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lcls_beamline_toolbox.xraywavetrace.beam1d as beam
import lcls_beamline_toolbox.xraywavetrace.optics1d as optics
import lcls_beamline_toolbox.xraywavetrace.beamline1d as beamline
import scipy.optimize as optimize
import copy
import scipy.spatial.transform as transform
from scipy.stats import qmc
from split_and_delay import SND

import torch
import gpytorch
import botorch

import warnings
warnings.filterwarnings("ignore")



from xopt import Xopt, Evaluator
from xopt.generators.bayesian import MOBOGenerator
from xopt.generators.bayesian import ExpectedImprovementGenerator, UpperConfidenceBoundGenerator
from xopt.resources.test_functions.tnk import evaluate_TNK, tnk_vocs
from xopt import VOCS
from xopt import Xopt

import math
import pandas as pd

In [73]:
def get_snd_outputs(inputs):
  """
  Study 1 Objective function. Takes an [n, 8] dim np array of
  [samples, ( t1_th1, t1_th2, chi1, chi2, t4_th1, t4_th2, chi1, chi2)].
  The entries lie in the uniform unit interval.
  They are scaled to lie in [-100e-6, +100e-6].
  Returns a torch tensor of dim [n, 2] of
  [samples, (do_sum_objective, IP_r_objective)]
  """
  inputs = inputs*200e-6 - 100e-6
  result = []

  for x in inputs:
    snd = SND(9500)
    x = np.array(x)

    snd.mvr_t1_th1(x[0])
    snd.mvr_t1_th2(x[1])
    snd.mvr_t1_chi1(x[2])
    snd.mvr_t1_chi2(x[3])
    snd.mvr_t4_th1(x[4])
    snd.mvr_t4_th2(x[5])
    snd.mvr_t4_chi1(x[6])
    snd.mvr_t4_chi2(x[7])

    snd.propagate_delay()

    dh1 = snd.get_t1_dh_sum()
    dd = snd.get_dd_sum()
    dh4 = snd.get_t4_dh_sum()
    do = snd.get_do_sum()
    my_IP_sum = snd.get_IP_sum()
    my_intensity = dh1 + dd + dh4 + do + my_IP_sum
    my_intensity *= (1.0 + np.random.randn() * 0.01)

    do_centroid = snd.get_IP_r()
    do_centroid *= (1.0 + np.random.randn() * 0.01)
    # do_centroid_x = snd.get_IP_cx()
    # do_centroid_y = snd.get_IP_cy()


    result.append( (np.log(do_centroid*1e6) - np.log(my_intensity*1e10)).item())

  return torch.tensor(result, dtype=torch.float)

In [74]:
def get_snd_outputs_detailed(inputs):
  """
  Study 1 Objective function. Takes an [n, 8] dim np array of
  [samples, ( t1_th1, t1_th2, chi1, chi2, t4_th1, t4_th2, chi1, chi2)].
  The entries lie in the uniform unit interval.
  They are scaled to lie in [-100e-6, +100e-6].
  Returns a torch tensor of dim [n, 2] of
  [samples, (do_sum_objective, IP_r_objective)]
  """
  inputs = inputs*200e-6 - 100e-6
  result = []

  for x in inputs:
    snd = SND(9500)
    x = np.array(x)

    snd.mvr_t1_th1(x[0])
    snd.mvr_t1_th2(x[1])
    snd.mvr_t1_chi1(x[2])
    snd.mvr_t1_chi2(x[3])
    snd.mvr_t4_th1(x[4])
    snd.mvr_t4_th2(x[5])
    snd.mvr_t4_chi1(x[6])
    snd.mvr_t4_chi2(x[7])

    snd.propagate_delay()

    dh1 = snd.get_t1_dh_sum()
    dd = snd.get_dd_sum()
    dh4 = snd.get_t4_dh_sum()
    do = snd.get_do_sum()
    my_IP_sum = snd.get_IP_sum()
    my_intensity = dh1 + dd + dh4 + do + my_IP_sum

    do_centroid = snd.get_IP_r()
    do_centroid_x = snd.get_IP_cx()
    do_centroid_y = snd.get_IP_cy()

    result.append([do_centroid*1e6, my_intensity])

  return torch.tensor(result, dtype=torch.float)

In [75]:
def eval_function(input_dict: dict) -> dict:
  x1, x2, x3, x4, x5, x6, x7, x8 = input_dict["x1"], input_dict["x2"], input_dict["x3"], input_dict["x4"], input_dict["x5"], input_dict["x6"], input_dict["x7"], input_dict["x8"]
  Xinp = np.expand_dims(np.array([x1, x2, x3, x4, x5, x6, x7, x8]), axis=0)
  output = get_snd_outputs(Xinp).squeeze()
  f = output.item()
  return {"f": f}

In [76]:
def run_turbo(eval_function=eval_function, init_samples=64, len_chain=150):
  low = 0.0
  high = 1.0
  n_init = 64
  vocs = VOCS(
      variables = {"x1": [low, high],
                  "x2": [low, high],
                  "x3": [low, high],
                  "x4": [low, high],
                  "x5": [low, high],
                  "x6": [low, high],
                  "x7": [low, high],
                  "x8": [low, high]
                  },
      objectives = {"f": "MINIMIZE"},
    )
  evaluator = Evaluator(function=eval_function)
  generator = ExpectedImprovementGenerator(
      vocs=vocs, turbo_controller="optimize"
  )
  X = Xopt(evaluator=evaluator, generator=generator, vocs=vocs)

  sampler = qmc.LatinHypercube(d=8)
  xs = sampler.random(n=n_init)
  init_samples = pd.DataFrame({f'x{i+1}': xs[:,i] for i in range(xs.shape[1])})
  X.evaluate_data(init_samples)

  X.generator.train_model()
  X.generator.turbo_controller.update_state(X.generator.data)
  X.generator.turbo_controller.get_trust_region(X.generator.model)

  for i in range(len_chain):
    if i % 10 == 0:
      print(f"Step: {i+1}")
    model = X.generator.train_model()
    trust_region = X.generator.turbo_controller.get_trust_region(generator.model)\
          .squeeze()
    scale_factor = X.generator.turbo_controller.length
    region_width = trust_region[1] - trust_region[0]
    best_value = X.generator.turbo_controller.best_value

    n_successes = X.generator.turbo_controller.success_counter
    n_failures = X.generator.turbo_controller.failure_counter

    acq = X.generator.get_acquisition(model)

    X.step()

  X.generator.turbo_controller

  return X

In [ ]:
X = run_turbo(eval_function=eval_function, init_samples=64, len_chain=150)

Step: 1


In [ ]:
y1 = X.generator.data["f"]
y1_mins = np.minimum.accumulate(y1)


idx = np.arange(len(y1_mins))
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 7))
ax0.plot(idx, y1_mins,'k')
ax0.grid()
ax0.set_xlabel("Sample Number")
ax0.set_ylabel("Objective")

ax1.plot(idx[-50:], y1_mins[-50:],'k')
ax1.grid()
ax1.set_ylabel("BPE ($\mu$m)");

In [ ]:
min_idx = np.argmin(X.generator.data["f"])
min_val = X.generator.data["f"][min_idx]
X_min = [X.generator.data["x1"][min_idx],
         X.generator.data["x2"][min_idx],
          X.generator.data["x3"][min_idx],
         X.generator.data["x4"][min_idx],
         X.generator.data["x5"][min_idx],
         X.generator.data["x6"][min_idx],
          X.generator.data["x7"][min_idx],
          X.generator.data["x8"][min_idx]
         ]
inputs = np.array(X_min)
inputs = inputs[np.newaxis,:]
outs = get_snd_outputs_detailed(inputs)
print("Optimum Inputs: ", X_min)
print("BPE: ", outs[0][0].item())
print("Intensity:", outs[0][1].item())
print(outs[0][0].item()/outs[0][1].item(), min_val)

In [ ]:
def plot_optima_from_generator_data(X, plot=True):
  """
  Takes Xopt object, returns and plots the (noise-less) data for
  beam position error and intensity.
  Args:
    X: Xopt object
    plot: boolean to plot or not
  Returns:
    tuple of beam position error sample and intensity sample histories.
  """
  features = ["x1",  "x2", "x3", "x4", "x5",  "x6", "x7", "x8"]
  points = X.generator.data[features].to_numpy()
  outs = get_snd_outputs_detailed(points).numpy()
  bpe_vals,  intensity_vals =   outs[:,0], outs[:,1]
  bpe_optima = np.minimum.accumulate(bpe_vals)
  intensity_optima = np.maximum.accumulate(intensity_vals)

  if plot:
    fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1, figsize=(7,10))
    ax1.plot(bpe_optima, 'k')
    ax1.scatter(np.arange(len(bpe_optima)), bpe_vals)
    ax1.set_xlabel("Sample Number")
    ax1.set_ylabel("BPE (microns)")


    ax2.plot(intensity_optima, 'k')
    ax2.scatter(np.arange(len(intensity_optima)), intensity_vals)
    ax2.set_xlabel("Sample Number")
    ax2.set_ylabel("Intensity")
    plt.show()

  return bpe_vals,  intensity_vals

In [ ]:
temp = plot_optima_from_generator_data(X)

In [ ]:
np.min(temp[0]), np.max(temp[1])

In [ ]:
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e4]
# Notes: Seems to over-value the BPE minimum compared to the Intensity maximum.
# BPE:  0.6
# Intensity: 38713

# BPE:  0.37
# Intensity: 28000

# BPE:  0.95
# Intensity: 28893

# BPE:  0.56
# Intensity: 66816

# BPE:  0.78
# Intensity: 59000

# BPE:  0.84
# Intensity: 26000
#==========================================================================
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e2]
#Seems to be over-valuing the BPE optimum at the cost of the intensity optimum
# BPE:  0.8
# Intensity: 28523

# BPE:  0.44
# Intensity: 38794

# BPE:  0.54
# Intensity: 15594

# BPE:  0.48
# Intensity: 29349

# BPE:  0.84
# Intensity: 15868

# BPE:  0.62
# Intensity: 66873

#==========================================================================
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e6]

# BPE:  0.29
# Intensity: 62596

# BPE:  1.2
# Intensity: 31887

# BPE:  2.3
# Intensity: 61435

# BPE:  0.31
# Intensity: 40029

# BPE:  0.76
# Intensity: 48991

# BPE:  0.21
# Intensity: 29005


#==========================================================================
# 1% noise, 64 init, 150 chain, weights = [1e6, 1e10]

# BPE:  0.88
# Intensity: 28331

# BPE:  0.13
# Intensity: 14723

# BPE:  0.41
# Intensity: 17541

In [ ]:
#Step 1. Track len_min and max for the TC
#Step 2. Instead of optimize setting, use the safety version of turbo.
#Step 3. Use the 'use_low_noise_prior' flag for the gp_constructor.